In [1]:
import pandas as pd
df = pd.read_excel("conversation.xlsx")
df = df[df['대분류'] == '비즈니스']
df = df[df['소분류']=='회의']
df = df[df['상황'] == '의견 교환하기']

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kou81\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [123]:
df

,대분류,소분류,상황,Set Nr.,발화자,원문,번역문
0,비즈니스,회의,의견 교환하기,1,A-1,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,비즈니스,회의,의견 교환하기,1,B-1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,비즈니스,회의,의견 교환하기,1,A-2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,비즈니스,회의,의견 교환하기,1,B-2,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,비즈니스,회의,의견 교환하기,2,A-1,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...
...,...,...,...,...,...,...,...
83031,비즈니스,회의,의견 교환하기,20758,B-2,그 분야에 대해서는 저도 조금 아는 부분이 있으니 제가 함께 준비하겠습니다.,I also know a little bit about that area so I'...
83032,비즈니스,회의,의견 교환하기,20759,A-1,"홍콩 파트너가 업무 협조를 너무 안 해주는데, 다른 업체로 바꾸는 게 어때요?",Our Hong Kong partners are not cooperating at ...
83033,비즈니스,회의,의견 교환하기,20759,B-1,"박람회에서 만난 홍콩 업체 명함이 있긴 한데, 신규 업체라는 점이 신경 쓰여요.",I have a business card from another firm in Ho...
83034,비즈니스,회의,의견 교환하기,20759,A-2,"오히려 신규 업체일수록 더 적극적으로 업무에 협조해 줄 수 있어요, 우선 그 업체에...","On the contrary, the new companies can be more..."


In [11]:
#inputText = "How is the market's reaction to the newly released product?"

In [132]:
inputText = df['번역문'][0]

In [23]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kou81\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [133]:
words = nltk.tokenize.word_tokenize(inputText)
word_tag = nltk.tag.pos_tag(words)
print(word_tag)

[('How', 'WRB'), ('is', 'VBZ'), ('the', 'DT'), ('market', 'NN'), ("'s", 'POS'), ('reaction', 'NN'), ('to', 'TO'), ('the', 'DT'), ('newly', 'RB'), ('released', 'VBN'), ('product', 'NN'), ('?', '.')]


In [127]:
# 크롤링 라이브러리 임포트
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep

driver = webdriver.Chrome('./webdriver/chromedriver.exe')
driver.implicitly_wait(2)

driver.get('https://papago.naver.com/')

In [135]:
# 정답데이터
inputWord = driver.find_element_by_name('txtSource')
inputWord.clear()
inputWord.send_keys(inputText)
sleep(2)

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
meaning = soup.select('#txtTarget span')
answer = meaning[0].text
print(answer)

새로 출시된 제품에 대한 시장의 반응은 어떻습니까?


In [129]:
# ()안에 있는 내용 삭제하고 ,(comma)가 있을 경우 첫 번째 값만 가져오도록 함수 정의
def seperateValue(word):
    print("입력된 문자 ==>" + word)
    word.replace(" ","") # 공백 제거
    
    tempList = word.split(")")
    word = ""
    for tempWord in tempList:
        word += tempWord
    
    tempList = word.split("(")
    word = ""
    for tempWord in tempList:
        if "→" not in tempWord:
            word += tempWord
    
    tempList = word.split(",")
    word = ""
    cnt = 0
    for tempWord in tempList:
        if cnt == 0:
            word += tempWord
            cnt = cnt + 1
    return word.replace(" ","")

#seperateValue("(일반적인) 시장[장] (→farmers’ market)")

In [136]:
wordResult = ""
tempText = ""
lastResult = ""
for word,pos in word_tag:
    print(word + " "+pos)
    # 입력 부분
    inputWord = driver.find_element_by_name('txtSource')
    inputWord.clear()
    inputWord.send_keys(word)
    print(word)
    if(word == '?'):
        print("마지막 word에 대한 Check ==>" + lastResult)
        lastResult = lastResult.strip()
        lastResult = lastResult.replace("이다","됩니까?")
        continue
    if(word == '.'):
        continue
    sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    if(pos == "NN"):
        meaning = soup.select('dd.dictionary_mean_box___26PlC strong.wordclass___m5RTy span')
        print(meaning[0].text)
        
        if(meaning[0].text == '명사'):
            meaning = soup.select('dd.dictionary_mean_box___26PlC span.dictionary_mean_text___30A6h')
            wordResult += seperateValue(meaning[0].text) + " "
            print(meaning[0].text)
        else:
            print(meaning[1].text)
        if tempText != "":
            wordResult += tempText
            tempText = ""
        print("==============================")
        
    # How 등 육하원칙의 동사에 대한 정의
    elif(pos == "WRB"):
        meaning = soup.select('#txtTarget span')
        print(meaning[0].text)
        lastResult += meaning[0].text + " "
        print("==============================")
        
    # 비동사에 대한 처리
    elif(pos == "VBZ"):
        meaning = soup.select('#txtTarget span')
        print(meaning[0].text)
        if(meaning[0].text == '이다'):
            lastResult += meaning[0].text
        print("==============================")
        
    # to에 대한 처리
    elif(pos == "TO"):
        meaning = soup.select('#txtTarget span')
        print(meaning[0].text)
        tempText += meaning[0].text + " 대한 "
        print("==============================")
        
    else:
        meaning = soup.select('#txtTarget span')
        print(meaning[0].text)
        wordResult += meaning[0].text + " "
        print("==============================")
        
print(wordResult+" "+lastResult)

How WRB
How
어떻게
is VBZ
is
이다
the DT
the
그
market NN
market
명사
입력된 문자 ==>(일반적인) 시장[장] (→farmers’ market)
(일반적인) 시장[장] (→farmers’ market)
's POS
's
의
reaction NN
reaction
명사
입력된 문자 ==>반응, 반작용
반응, 반작용
to TO
to
에
the DT
the
그
newly RB
newly
새로
released VBN
released
방출된
product NN
product
명사
입력된 문자 ==>생산물, 상품, 제품 (→end product, gross national product)
생산물, 상품, 제품 (→end product, gross national product)
? .
?
마지막 word에 대한 Check ==>어떻게 이다
그 일반적인시장[장] 의 반응 그 새로 방출된 생산물 에 대한  어떻게 됩니까?


In [42]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('./wiki/wiki.model')

In [44]:
model.wv.most_similar(positive=["How","is"])

[('Why', 0.8973870873451233),
 ('Does', 0.8674536943435669),
 ('They', 0.8647955060005188),
 ('will', 0.8571343421936035),
 ('your', 0.8469832539558411),
 ('like', 0.8461585640907288),
 ('there', 0.8440003395080566),
 ('what', 0.8398547768592834),
 ('Then', 0.8386229276657104),
 ('When', 0.8380194306373596)]

In [71]:
data = seperateValue("(일반적인) 시장[장] (→farmers’ market)","(")
print(data)

일반적인) 시장[장] →farmers’ market)


In [70]:
data = seperateValue(data,")")
print(data)

일반적인 시장[장] →farmers’ market


입력된 문자 ==>(일반적인) 시장[장] (→farmers’ market)
일반적인 시장[장] 


In [115]:
print("어떻게 이다".replace("이다","됩니까?"))

어떻게 됩니까?
